In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [3]:
df[df['press'] == 'PRNewswire']['url'].iloc[1]

'https://www.prnewswire.com/news-releases/idc-futurescape-worldwide-utilities-2018-predictions-300553415.html'

In [4]:
PRNewswire = df[df['press'] == 'PRNewswire']
PRNewswire

,source,author,title,description,url,urlToImage,publishedAt,content,press
42,"{'id': None, 'name': 'PRNewswire'}","MGT Capital Investments, Inc.",MGT Capital Reports Third Quarter Statement of...,"DURHAM, N.C., Nov. 9, 2017 /PRNewswire/ -- MGT...",https://www.prnewswire.com/news-releases/mgt-c...,https://mma.prnewswire.com/media/601375/MGT_CA...,2017-11-09T22:00:00Z,"Revenues from Bitcoin mining were $515,000, up...",PRNewswire
43,"{'id': None, 'name': 'PRNewswire'}",Reportlinker,IDC FutureScape: Worldwide Utilities 2018 Pred...,"NEW YORK, Nov. 9, 2017 /PRNewswire/ -- This ID...",https://www.prnewswire.com/news-releases/idc-f...,https://content.prnewswire.com/images/prn_face...,2017-11-09T22:00:00Z,"NEW YORK, Nov. 9, 2017 /PRNewswire/ -- This ID...",PRNewswire
191,"{'id': None, 'name': 'PRNewswire'}",Safehaven.com,The Elite Team Tackling the World's Biggest Se...,"LONDON, November 10, 2017 /PRNewswire/ -- This...",https://www.prnewswire.com/news-releases/the-e...,https://content.prnewswire.com/images/prn_face...,2017-11-10T13:00:00Z,Every industry is reeling from this battle. Th...,PRNewswire
216,"{'id': None, 'name': 'PRNewswire'}",FinancialBuzz.com,Total Cryptocurrency Market Cap Breaks New Rec...,"NEW YORK, November 10, 2017 /PRNewswire/ -- Th...",https://www.prnewswire.com/news-releases/total...,https://content.prnewswire.com/images/prn_face...,2017-11-10T14:00:00Z,The total cryptocurrency market capitalization...,PRNewswire
218,"{'id': None, 'name': 'PRNewswire'}",FinancialBuzz.com,Bitcoin Price Rally Continues as a New Split i...,"NEW YORK, November 10, 2017 /PRNewswire/ -- Bi...",https://www.prnewswire.com/news-releases/bitco...,https://content.prnewswire.com/images/prn_face...,2017-11-10T14:00:00Z,Bitcoin price jumped on Wednesday after the de...,PRNewswire
...,...,...,...,...,...,...,...,...,...
744077,"{'id': None, 'name': 'PRNewswire'}",NaN,Unicoin Makes History with $335M Thailand Luxu...,"NEW YORK, Sept. 29, 2023 /PRNewswire/ -- Unico...",https://www.prnewswire.com/news-releases/unico...,https://mma.prnewswire.com/media/2223728/Eden_...,2023-09-29T16:14:00Z,"NEW YORK, Sept. 29, 2023 /PRNewswire/ -- Unico...",PRNewswire
744095,"{'id': None, 'name': 'PRNewswire'}",NaN,Bitcoin Price Climbs 3% on US Shutdown and Yie...,"NEW YORK, Sept. 29, 2023 /PRNewswire/ -- The B...",https://www.prnewswire.com/news-releases/bitco...,https://mma.prnewswire.com/media/2235370/Bitco...,2023-09-29T16:48:00Z,"NEW YORK, Sept. 29, 2023 /PRNewswire/ -- The B...",PRNewswire
744117,"{'id': None, 'name': 'PRNewswire'}",NaN,SHAREHOLDER ALERT: Pomerantz Law Firm Reminds ...,"NEW YORK, Sept. 29, 2023 /PRNewswire/ -- Pomer...",https://www.prnewswire.com/news-releases/share...,https://mma.prnewswire.com/media/1486339/Pomer...,2023-09-29T17:24:00Z,"NEW YORK, Sept. 29, 2023 /PRNewswire/ -- Pomer...",PRNewswire
744205,"{'id': None, 'name': 'PRNewswire'}",NaN,World Market for Luxury Goods: Green Initiativ...,"DUBLIN, Sept. 29, 2023 /PRNewswire/ -- The ""Wo...",https://www.prnewswire.com/news-releases/world...,https://mma.prnewswire.com/media/539438/Resear...,2023-09-29T21:30:00Z,"DUBLIN, Sept. 29, 2023 /PRNewswire/ -- The ""Wo...",PRNewswire


In [15]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
                response = requests.get(url, headers=headers)
                soup = BeautifulSoup(response.text, "html.parser")
                content = soup.select(".col-sm-offset-1")
                cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
                contents.append(cleaned_content)
        except:
                contents.append('')

    return contents

In [26]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")

            # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
            classes = set()
            for tag in soup.find_all(True, {'class': True}):
                for class_name in tag['class']:
                    if re.match(r'col-\w+-offset-1', class_name):
                        classes.add(class_name)

            # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
            content = []
            for class_name in classes:
                content.extend(soup.select(f".{class_name}"))
            
            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
            contents.append(cleaned_content)
        except:
            contents.append('')

    return contents


In [21]:
head = PRNewswire.tail()

In [29]:
# Adding the content to the DataFrame
PRNewswire['content_full'] = get_contents(PRNewswire)
PRNewswire

  1%|          | 182/16565 [06:48<7:09:55,  1.57s/it] 

In [24]:
head['url'].iloc[0]

'https://www.prnewswire.com/news-releases/unicoin-makes-history-with-335m-thailand-luxury-resort-acquisition-the-largest-ever-property-for-crypto-deal-301943190.html'

In [28]:
head['content_full'].iloc[0]

'\nNEW YORK, Sept. 29, 2023 /PRNewswire/ -- Unicoin, an assets-backed cryptocurrency that addresses the extreme volatility of the crypto market, has signed a landmark $335 million agreement to purchase Eden Grand Resort, a development project for a 64,000 square meter property and luxury resort in Chonburi, Thailand - the largest-ever real estate acquisition made with cryptocurrency.\xa0\n\n\nContinue Reading\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nUnicoin Makes History with $335M Thailand Luxury Resort Acquisition, the Largest-Ever Property-for-Crypto Deal\n\n\n\n\n\n\n\nThe property, in addition to the underlying land, is proposed to have a 6-story building plus 150 2- and 3-bedroom villas constructed and developed as a luxury resort. The purchase price shall be paid in unicoins. The acquisition is structured pursuant to Unicoin\'s program of acquiring real estate assets at 140% of their appraised value for unicoins. Unicoin Inc. agreed to pay Mr. Adnan 671,206,755 unicoins for the acquis

In [28]:
content100['content_full']

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [14]:
Bitcoinist[Bitcoinist['content_full'] == '']

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
57792,"{'id': None, 'name': 'Bitcoinist'}",Bitcoinist.net,5 Interesting Token Sale Projects to Look at i...,"With so many ICOs launching all the time, how ...",http://bitcoinist.com/5-interesting-token-sale...,http://bitcoinist.com/wp-content/uploads/2017/...,2018-02-21T20:45:26Z,"Bitcoinist.net · February 21, 2018 · 3:45 pm W...",Bitcoinist,
67530,"{'id': None, 'name': 'Bitcoinist'}",Bitcoinist.net,Specialist Trading Token ThinkCoin Launches fo...,ThinkCoin has launched a new digital trading t...,http://bitcoinist.com/specialist-trading-token...,http://bitcoinist.com/wp-content/uploads/2018/...,2018-03-15T16:30:32Z,"Bitcoinist.net · March 15, 2018 · 12:30 pm Thi...",Bitcoinist,
180272,"{'id': None, 'name': 'Bitcoinist'}",Esther Kim,Scam Alert: Bitconnect ‘2.0’ Claims Binance Li...,The return of notorious cryptocurrency ponzi s...,https://bitcoinist.com/bitconnect-2-scam-binan...,https://bitcoinist.com/wp-content/uploads/2019...,2019-05-20T09:00:55Z,The return of notorious cryptocurrency ponzi s...,Bitcoinist,
193036,"{'id': None, 'name': 'Bitcoinist'}",Anja van Oosterhout,Police Raid Tron’s Beijing Office As CEO Battl...,Blockchain network Tron is at the center of co...,https://bitcoinist.com/police-raid-trons-beiji...,https://bitcoinist.com/wp-content/uploads/2019...,2019-07-08T12:24:16Z,Blockchain network Tron is at the center of co...,Bitcoinist,
194147,"{'id': None, 'name': 'Bitcoinist'}",Julio Gil-Pulgar,Major Australian Supermarket Chain Now Accepts...,"Slowly but surely, bitcoin is gathering moment...",https://bitcoinist.com/major-australian-superm...,https://bitcoinist.com/wp-content/uploads/2019...,2019-07-11T20:54:58Z,"Slowly but surely, bitcoin is gathering moment...",Bitcoinist,
212809,"{'id': None, 'name': 'Bitcoinist'}",Christine Vasileva,Redditors Shilling NANO Now Face $170 Million ...,A time of reckoning is coming for the NANO cry...,https://bitcoinist.com/redditors-shilling-nano...,https://bitcoinist.com/wp-content/uploads/2019...,2019-10-08T12:30:37Z,A time of reckoning is coming for the NANO cry...,Bitcoinist,
215692,"{'id': None, 'name': 'Bitcoinist'}",Julio Gil-Pulgar,"HTC Launches Exodus 1, ‘To Make The Leap To Bi...",The advent of Bitcoin is fueling the evolution...,https://bitcoinist.com/htc-launches-exodus-1-b...,https://bitcoinist.com/wp-content/uploads/2018...,2019-10-20T19:00:35Z,The advent of Bitcoin is fueling the evolution...,Bitcoinist,
220543,"{'id': None, 'name': 'Bitcoinist'}",Martin Young,Chinese E-Commerce Giant Alibaba Offers Free B...,"Yes, you did read that right. The company that...",https://bitcoinist.com/alibaba-offers-free-bit...,https://bitcoinist.com/wp-content/uploads/2019...,2019-11-12T05:17:12Z,"Yes, you did read that right. The company that...",Bitcoinist,
230903,"{'id': None, 'name': 'Bitcoinist'}",Christine Vasileva,CME Bitcoin Options Launch on “Great Anticipat...,The CME Bitcoin options market has sparked gre...,https://bitcoinist.com/cme-bitcoin-options-lau...,https://bitcoinist.com/wp-content/uploads/2019...,2020-01-13T17:01:15Z,The CME Bitcoin options market has sparked gre...,Bitcoinist,
232646,"{'id': None, 'name': 'Bitcoinist'}",Osato Avan-Nomayo,"Bitcoin Puzzle Worth 2.1 BTC STILL Unsolved, F...",With its 2.1 bitcoin (BTC) puzzle still unsolv...,https://bitcoinist.com/bitcoin-puzzle-worth-2-...,https://bitcoinist.com/wp-content/uploads/2020...,2020-01-23T23:00:30Z,With its 2.1 bitcoin (BTC) puzzle still unsolv...,Bitcoinist,


In [37]:
Bitcoinist['content_full'] = get_contents(content100)
Bitcoinist

100%|██████████| 100/100 [00:00<00:00, 5821.46it/s]


source                            {'id': None, 'name': 'newsBTC'}
author                                             Dalmas Ngetich
title           Tron Price Analysis: TRX Risk Dropping to Nint...
description     Tron prices bullish but under pressure hours a...
url             https://www.newsbtc.com/2019/02/11/tron-price-...
urlToImage      https://www.newsbtc.com/wp-content/uploads/201...
publishedAt                                  2019-02-11T17:00:22Z
content         Tron prices bullish but under pressure hours a...
press                                                     newsBTC
content_full    [, , , , , , , , , , , , , , , , , , , , , , ,...
Name: 158978, dtype: object

In [19]:
empty_rows = newsBTC[newsBTC['content_full'] == '']

empty_rows['url'].iloc[100]

'https://www.newsbtc.com/2019/02/04/crypto-researcher-waning-bitcoin-interest-twitter-screams-bearish/'

In [16]:
Bitcoinist.to_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\Bitcoinist_7.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content